In [28]:
import pandas as pd
import matplotlib.pyplot as plt
import os

In [29]:
folder_path = "/Users/oas/Downloads/benchmarking_results"
# folder_path = "/Users/oas/Documents/VOD/vod/benchmarking_results"
files = [
    os.path.join(folder_path, file)
    for file in os.listdir(folder_path)
    if os.path.isfile(os.path.join(folder_path, file))
]
files.sort()  # Sort the files by name
newest_file = files[-1]  # Pick the newest
print(newest_file)

/Users/oas/Downloads/benchmarking_results/2024-02-13-01-16-35.csv


In [30]:
df_results = pd.read_csv(newest_file)
df_results.shape

(324, 16)

In [31]:
df_results.head()

,Unnamed: 0,Dataset,Index,Index parameters.,Server startup speed (s),Index build speed (s),Search speed avg. (ms),Search speed p95 (ms),Recall avg,Recall@1,Recall@10,Recall@100,ingesting_max,ingesting_mean,benchmarking_max,benchmarking_mean
0,0,sift-128-euclidean,"Qdrant IVF, n_partition=100, None, L2",error: Traceback (most recent call last):\n F...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,1,sift-128-euclidean,"Qdrant IVF, n_partition=100, None, DOT",error: Traceback (most recent call last):\n F...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,2,sift-128-euclidean,"Qdrant IVF, n_partition=1000, None, L2",error: Traceback (most recent call last):\n F...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,3,sift-128-euclidean,"Qdrant IVF, n_partition=1000, None, DOT",error: Traceback (most recent call last):\n F...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,4,sift-128-euclidean,"Qdrant IVF, n_partition=10000, None, L2",error: Traceback (most recent call last):\n F...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [32]:
df_results = df_results.query("Index != 'None'")  # ignore failed runs
df_results = df_results.query("benchmarking_mean != -1")  # ignore failed runs

In [33]:
df_results["index_prov"] = df_results.Index.str.split(" ").apply(lambda l: l[1])

In [34]:
df_results = df_results.dropna()

In [35]:
df_results.columns

Index(['Unnamed: 0', 'Dataset', 'Index', 'Index parameters.',
       'Server startup speed (s)', 'Index build speed (s)',
       'Search speed avg. (ms)', 'Search speed p95 (ms)', 'Recall avg',
       'Recall@1', 'Recall@10', 'Recall@100', 'ingesting_max',
       'ingesting_mean', 'benchmarking_max', 'benchmarking_mean',
       'index_prov'],
      dtype='object')

In [36]:
df_results.shape

(162, 17)

In [37]:
# df_results[df_results.isna()]

In [42]:
import plotly.express as px
import plotly.graph_objects as go

# Your existing DataFrame and plot
fig = px.scatter(
    df_results,
    x="Recall avg",
    y="Search speed avg. (ms)",
    hover_data=["Index","ingesting_max","benchmarking_mean"],
    color="index_prov",
    size=df_results["benchmarking_mean"],
)

# Adding a similar scatter plot but with semi-transparency and different size mapping
fig.add_trace(
    go.Scatter(
        x=df_results["Recall avg"],
        y=df_results["Search speed avg. (ms)"],
        mode="markers",
        marker=dict(
            size=df_results["ingesting_max"] / 25,  # Size based on 'ingesting_max' column
            opacity=0.1,  # Semi-transparent markers
            # color=fig.data[0].marker.color,  # Use the same color as the first plot
            # line=dict(color="MediumPurple", width=2),
        ),
        hoverinfo="text",  # You can customize hover info as needed
        name="Ingesting max",
    )
)

fig.show()